## Transfer Learning and Knowledge Distillation
 In this notebook, the goal is to apply knowledge distillation to a student other than the master but with comparable complexity. To speed up the teacher's training process we will use a pre-trained model.

In [1]:
#All the imports
import tensorflow as tf
from BANEnsemble import BANEnsemble
from knowledge_distillation import distill_knowledge, ban
from WideResNet import WideResidualNetwork

In [2]:
# Connection with the dataset in drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
! unzip /content/drive/My\ Drive/datasets/bird_species.zip 

Output streaming troncato alle ultime 5000 righe.
  inflating: train/SUPERB STARLING/069.jpg  
  inflating: train/SUPERB STARLING/070.jpg  
  inflating: train/SUPERB STARLING/071.jpg  
  inflating: train/SUPERB STARLING/072.jpg  
  inflating: train/SUPERB STARLING/073.jpg  
  inflating: train/SUPERB STARLING/074.jpg  
  inflating: train/SUPERB STARLING/075.jpg  
  inflating: train/SUPERB STARLING/076.jpg  
  inflating: train/SUPERB STARLING/077.jpg  
  inflating: train/SUPERB STARLING/078.jpg  
  inflating: train/SUPERB STARLING/079.jpg  
  inflating: train/SUPERB STARLING/080.jpg  
  inflating: train/SUPERB STARLING/081.jpg  
  inflating: train/SUPERB STARLING/082.jpg  
  inflating: train/SUPERB STARLING/083.jpg  
  inflating: train/SUPERB STARLING/084.jpg  
  inflating: train/SUPERB STARLING/085.jpg  
  inflating: train/SUPERB STARLING/086.jpg  
  inflating: train/SUPERB STARLING/087.jpg  
  inflating: train/SUPERB STARLING/088.jpg  
  inflating: train/SUPERB STARLING/089.jpg  
  inf

## Preprocessing and Augmentation
In order to maintain consistency with previous tests, the preprocessing and image augmentation will be the same

In [4]:
BATCH_SIZE = 32
IMAGE_SIZE = (96, 96)
data_path = '/content'

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=0.05        )  
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    )

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
train_generator = train_datagen.flow_from_directory(
        data_path+'/train',
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        )
validation_generator = valid_datagen.flow_from_directory(
        data_path+'/valid',
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        )
test_generator = test_datagen.flow_from_directory(
        data_path+'/test',
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 31316 images belonging to 225 classes.
Found 1125 images belonging to 225 classes.
Found 1125 images belonging to 225 classes.


In [ ]:
base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(96, 96, 3), classes=225)


9412608/9406464 [==============================] - 0s 0us/step


In [6]:
teacher_model = tf.keras.models.load_model('mobile_net.h5')

In [ ]:
base_model.trainable=False


In [ ]:
inputs = tf.keras.layers.Input(shape=(96, 96, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(225)(x)
x = tf.keras.layers.Activation('softmax')(x)

teacher_model = tf.keras.models.Model(inputs, x)

teacher_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
teacher_model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_10  (None, 1280)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 225)               288225    
_________________________________________________________________
activation_12 (Activation)   (None, 225)               0         
Total params: 2,546,209
Trainable params: 288,225
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:

history = teacher_model.fit(train_generator, 
                            batch_size=BATCH_SIZE,
                            validation_data=validation_generator,
                            steps_per_epoch=train_generator.samples//BATCH_SIZE,
                            validation_steps=validation_generator.samples//BATCH_SIZE,
                            epochs=15)

Epoch 1/15
 22/978 [..............................] - ETA: 1:46 - loss: 0.1592 - accuracy: 0.9517

KeyboardInterrupt: ignored

In [7]:
#teacher_model.save('mobile_net.h5')
teacher_model.evaluate(test_generator)

36/36 [==============================] - 2s 44ms/step - loss: 0.2924 - accuracy: 0.9182


[0.2923920154571533, 0.9182222485542297]

In [ ]:
base_model.trainable=True

In [ ]:
print(f"The base_model has: {len(base_model.layers)} layers")

The base_model has: 155 layers


In [ ]:
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable=False

In [ ]:
teacher_model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_10  (None, 1280)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 225)               288225    
_________________________________________________________________
activation_12 (Activation)   (None, 225)               0         
Total params: 2,546,209
Trainable params: 288,225
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
calls = [
             tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
             tf.keras.callbacks.ModelCheckpoint('densenet_teacher.h5', save_best_only=True, save_weights_only=True)
]
history = teacher_model.fit(train_generator, 
                            batch_size=BATCH_SIZE,
                            validation_data=validation_generator,
                            steps_per_epoch=train_generator.samples//BATCH_SIZE,
                            validation_steps=validation_generator.samples//BATCH_SIZE,
                            epochs=100,
                            callbacks=calls)

Epoch 1/100
978/978 [==============================] - 123s 126ms/step - loss: 0.1489 - accuracy: 0.9530 - val_loss: 0.3981 - val_accuracy: 0.8964
Epoch 2/100
978/978 [==============================] - 122s 125ms/step - loss: 0.1462 - accuracy: 0.9527 - val_loss: 0.4118 - val_accuracy: 0.8946
Epoch 3/100
978/978 [==============================] - 122s 125ms/step - loss: 0.1405 - accuracy: 0.9560 - val_loss: 0.4267 - val_accuracy: 0.8893
Epoch 4/100
978/978 [==============================] - 121s 124ms/step - loss: 0.1353 - accuracy: 0.9567 - val_loss: 0.4297 - val_accuracy: 0.8955
Epoch 5/100
978/978 [==============================] - 121s 124ms/step - loss: 0.1386 - accuracy: 0.9555 - val_loss: 0.4204 - val_accuracy: 0.8929
Epoch 6/100
978/978 [==============================] - 121s 124ms/step - loss: 0.1285 - accuracy: 0.9593 - val_loss: 0.4252 - val_accuracy: 0.8946


In [ ]:
teacher_model.evaluate(test_generator)

36/36 [==============================] - 2s 45ms/step - loss: 0.3480 - accuracy: 0.9111


[0.34798598289489746, 0.9111111164093018]

In [ ]:
teacher_model.layers[-4].trainable= True
teacher_model.summary()
for layer in teacher_model.layers[-4].layers[:fine_tune_at]:
  layer.trainable=False
teacher_model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_10  (None, 1280)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 225)               288225    
_________________________________________________________________
activation_12 (Activation)   (None, 225)               0         
Total params: 2,546,209
Trainable params: 2,512,097
Non-trainable params: 34,112
_________________________________________________________________
Model: "functional_11"
_________________________________________________________________
Layer (type)   

In [20]:
student_network = tf.keras.models.Sequential(
    [
     WideResidualNetwork(225, 16, 4, includeActivation=False),
     tf.keras.layers.Activation('softmax')
    ]
)
student_network.build(input_shape=(None, 96, 96, 3))
student_network.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
wide_residual_network_12 (Wi (None, 225)               2836845   
_________________________________________________________________
activation_25 (Activation)   (None, 225)               0         
Total params: 2,836,845
Trainable params: 2,833,639
Non-trainable params: 3,206
_________________________________________________________________


In [24]:
student_network.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
BATCH_SIZE=32
callbacks = [
             tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='loss'),
             tf.keras.callbacks.ModelCheckpoint('student.h5', save_weights_only=True, save_best_only=True)
]
fit_args = dict(
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//BATCH_SIZE,
    steps_per_epoch=train_generator.samples//BATCH_SIZE,
    callbacks = callbacks

)
distill_knowledge(teacher_model, student_network, train_generator, fit_args=fit_args)

Epoch 1/100
978/978 [==============================] - 222s 227ms/step - loss: 5.0702 - accuracy: 0.0336 - val_loss: 4.7325 - val_accuracy: 0.0643
Epoch 2/100
978/978 [==============================] - 226s 231ms/step - loss: 4.3382 - accuracy: 0.0993 - val_loss: 4.2629 - val_accuracy: 0.1455
Epoch 3/100
978/978 [==============================] - 226s 231ms/step - loss: 3.6704 - accuracy: 0.1884 - val_loss: 3.7061 - val_accuracy: 0.2205
Epoch 4/100
978/978 [==============================] - 226s 231ms/step - loss: 3.0534 - accuracy: 0.3078 - val_loss: 2.4625 - val_accuracy: 0.3991
Epoch 5/100
978/978 [==============================] - 226s 231ms/step - loss: 2.5566 - accuracy: 0.4071 - val_loss: 2.7652 - val_accuracy: 0.3714
Epoch 6/100
978/978 [==============================] - 226s 231ms/step - loss: 2.1778 - accuracy: 0.4932 - val_loss: 2.7952 - val_accuracy: 0.4429
Epoch 7/100
978/978 [==============================] - 227s 232ms/step - loss: 1.9155 - accuracy: 0.5573 - val_loss: 2

In [31]:
teacher_model.evaluate(test_generator)
student_network.load_weights('student.h5')
student_network.evaluate(test_generator)


36/36 [==============================] - 2s 60ms/step - loss: 0.2301 - accuracy: 0.9333


[0.23009125888347626, 0.9333333373069763]